In [1]:
import sys
from pathlib import Path
sys.path.insert(0, Path(sys.path[0]).parent.as_posix())
from lxml import etree
import pickle
from cent import between_cent, degree_cent, eigen_cent
import logging
from pathlib import Path
from datetime import datetime

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s [%(levelname)s]: %(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S'
                )
logger = logging.getLogger(__name__)
file_handler = logging.FileHandler('cent_cal.log')
file_handler.setLevel(logging.DEBUG)
file_formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')
file_handler.setFormatter(file_formatter)
logger.addHandler(file_handler)

In [2]:
def save_data(nodes, edges, file_path):
    with file_path.open('wb') as f:
        pickle.dump({'nodes': nodes, 'edges': edges}, f)        
    

def load_data(file_path):
    with file_path.open('rb') as f:
        data = pickle.load(f)
    return data['nodes'], data['edges']

In [3]:
graph_path = Path.cwd().parent.joinpath("data", 'graph_nodes_edges.pkl')

if  graph_path.exists():
    logger.info("Loading nodes and edges from saved file.")
    nodes, edges = load_data(graph_path)
else:
    file_path = Path.cwd().parent.joinpath("data", "graph_metric.graphml").as_posix()
    logger.info("Parsing nodes and edges from GraphML.")
    # generate nodes and edges from graphml
    now = datetime.now()
    nodes, edges = parse_graphml_in_chunks(file_path)
    logger.info(f"Time spent for node loading from graphml is: {datetime.now() - now}")
    save_data(nodes, edges, graph_path)


2024-11-08 11:38:12 [INFO]: Loading nodes and edges from saved file.


In [4]:
import json

def str_to_json(escaped_json_str):
    try:
        clean_str = escaped_json_str.replace('\\"', '"')
        return json.loads(clean_str)
    except ValueError as e:
        print(f"Error parsing JSON: {e}")
        return None

In [5]:
# Filter nodes that have the 'severity' attribute
nodes_with_severity = {node_id: node for node_id, node in nodes.items() if 'type' in node and node['type'] == "CVE" and str_to_json(node["value"])['cve'] !=[]}
# nodes_with_severity = {node_id: node for node_id, node in nodes.items() if 'type' in node and str_to_json(node["value"])["cve"]!=[]}

In [6]:
# Using list slicing to print the first two items
for node_id, node in list(nodes_with_severity.items())[:10]:
    print(node_id, node)

n14293237 {'labels': ':AddedValue', 'id': 'org.keycloak:keycloak-core:3.4.1.Final:CVE', 'type': 'CVE', 'value': '{\\"cve\\":[{\\"cwe\\":\\"[CWE-267]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2019-10170\\"},{\\"cwe\\":\\"[CWE-79]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2022-0225\\"},{\\"cwe\\":\\"[CWE-79]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1697\\"},{\\"cwe\\":\\"[CWE-547,CWE-798]\\",\\"severity\\":\\"CRITICAL\\",\\"name\\":\\"CVE-2019-14837\\"},{\\"cwe\\":\\"[CWE-306]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2021-20262\\"},{\\"cwe\\":\\"[CWE-1021]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1728\\"},{\\"cwe\\":\\"[CWE-285,CWE-287]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2018-14637\\"},{\\"cwe\\":\\"[CWE-276]\\",\\"severity\\":\\"LOW\\",\\"name\\":\\"UNKNOWN\\"},{\\"cwe\\":\\"[CWE-285]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-10686\\"},{\\"cwe\\":\\"[CWE-20]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-202

In [8]:
example_node= {"n0": 
               {'labels': ':AddedValue', 
                 'id': 'org.keycloak:keycloak-core:3.4.1.Final:CVE', 'type': 'CVE', 'value': '{\\"cve\\":[{\\"cwe\\":\\"[CWE-267]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2019-10170\\"},{\\"cwe\\":\\"[CWE-79]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2022-0225\\"},{\\"cwe\\":\\"[CWE-79]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1697\\"},{\\"cwe\\":\\"[CWE-547,CWE-798]\\",\\"severity\\":\\"CRITICAL\\",\\"name\\":\\"CVE-2019-14837\\"},{\\"cwe\\":\\"[CWE-306]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2021-20262\\"},{\\"cwe\\":\\"[CWE-1021]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1728\\"},{\\"cwe\\":\\"[CWE-285,CWE-287]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2018-14637\\"},{\\"cwe\\":\\"[CWE-276]\\",\\"severity\\":\\"LOW\\",\\"name\\":\\"UNKNOWN\\"},{\\"cwe\\":\\"[CWE-285]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-10686\\"},{\\"cwe\\":\\"[CWE-20]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2020-1714\\"},{\\"cwe\\":\\"[CWE-287,CWE-841]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"UNKNOWN\\"},{\\"cwe\\":\\"[CWE-613]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1724\\"},{\\"cwe\\":\\"[CWE-835]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2018-10912\\"},{\\"cwe\\":\\"[CWE-287]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-27838\\"},{\\"cwe\\":\\"[CWE-287,CWE-841]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2023-0105\\"},{\\"cwe\\":\\"[CWE-200,CWE-755]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1744\\"},{\\"cwe\\":\\"[CWE-295,CWE-345]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2019-3875\\"},{\\"cwe\\":\\"[CWE-601]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"UNKNOWN\\"},{\\"cwe\\":\\"[CWE-200,CWE-532]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-1698\\"},{\\"cwe\\":\\"[CWE-863]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2022-1466\\"},{\\"cwe\\":\\"[CWE-200]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2019-14820\\"},{\\"cwe\\":\\"[CWE-295]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"UNKNOWN\\"},{\\"cwe\\":\\"[CWE-250]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2020-27826\\"},{\\"cwe\\":\\"[CWE-377]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2021-20202\\"},{\\"cwe\\":\\"[CWE-330,CWE-341]\\",\\"severity\\":\\"CRITICAL\\",\\"name\\":\\"CVE-2020-1731\\"},{\\"cwe\\":\\"[CWE-80]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2022-0225\\"},{\\"cwe\\":\\"[CWE-645]\\",\\"severity\\":\\"LOW\\",\\"name\\":\\"CVE-2024-1722\\"},{\\"cwe\\":\\"[CWE-200]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2019-3868\\"},{\\"cwe\\":\\"[CWE-287]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2021-3632\\"},{\\"cwe\\":\\"[CWE-295]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-35509\\"},{\\"cwe\\":\\"[CWE-79]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"UNKNOWN\\"},{\\"cwe\\":\\"[CWE-601,CWE-918]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2020-10770\\"},{\\"cwe\\":\\"[CWE-20,CWE-352]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2019-10199\\"},{\\"cwe\\":\\"[CWE-347]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2019-10201\\"},{\\"cwe\\":\\"[CWE-284,CWE-863]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2023-0091\\"},{\\"cwe\\":\\"[CWE-295]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2023-1664\\"},{\\"cwe\\":\\"[CWE-602]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2017-12161\\"},{\\"cwe\\":\\"[CWE-116,CWE-20,CWE-79]\\",\\"severity\\":\\"CRITICAL\\",\\"name\\":\\"CVE-2021-20195\\"},{\\"cwe\\":\\"[CWE-22,CWE-552]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2021-3856\\"},{\\"cwe\\":\\"[CWE-269,CWE-916]\\",\\"severity\\":\\"HIGH\\",\\"name\\":\\"CVE-2020-14389\\"},{\\"cwe\\":\\"[CWE-20]\\",\\"severity\\":\\"MODERATE\\",\\"name\\":\\"CVE-2021-3754\\"}]}'
             },
              }

In [9]:
str_to_json(example_node['n0']['value'])['cve']

[{'cwe': '[CWE-267]', 'severity': 'HIGH', 'name': 'CVE-2019-10170'},
 {'cwe': '[CWE-79]', 'severity': 'MODERATE', 'name': 'CVE-2022-0225'},
 {'cwe': '[CWE-79]', 'severity': 'MODERATE', 'name': 'CVE-2020-1697'},
 {'cwe': '[CWE-547,CWE-798]',
  'severity': 'CRITICAL',
  'name': 'CVE-2019-14837'},
 {'cwe': '[CWE-306]', 'severity': 'MODERATE', 'name': 'CVE-2021-20262'},
 {'cwe': '[CWE-1021]', 'severity': 'MODERATE', 'name': 'CVE-2020-1728'},
 {'cwe': '[CWE-285,CWE-287]', 'severity': 'HIGH', 'name': 'CVE-2018-14637'},
 {'cwe': '[CWE-276]', 'severity': 'LOW', 'name': 'UNKNOWN'},
 {'cwe': '[CWE-285]', 'severity': 'MODERATE', 'name': 'CVE-2020-10686'},
 {'cwe': '[CWE-20]', 'severity': 'HIGH', 'name': 'CVE-2020-1714'},
 {'cwe': '[CWE-287,CWE-841]', 'severity': 'MODERATE', 'name': 'UNKNOWN'},
 {'cwe': '[CWE-613]', 'severity': 'MODERATE', 'name': 'CVE-2020-1724'},
 {'cwe': '[CWE-835]', 'severity': 'MODERATE', 'name': 'CVE-2018-10912'},
 {'cwe': '[CWE-287]', 'severity': 'MODERATE', 'name': 'CVE-20

## process freshness value

In [21]:
# Filter nodes that have the 'severity' attribute
nodes_with_freshness = {node_id: node for node_id, node in nodes.items() if 'type' in node and node['type'] == "FRESHNESS" and str_to_json(node["value"])['freshness'] !={}}

In [22]:
for node_id, node in list(nodes_with_freshness.items())[:10]:
    print(node_id, str_to_json(node["value"]))

n29576356 {'freshness': {'numberMissedRelease': '7', 'outdatedTimeInMs': '3795765000'}}
n29576358 {'freshness': {'numberMissedRelease': '2', 'outdatedTimeInMs': '11941344000'}}
n29576360 {'freshness': {'numberMissedRelease': '9', 'outdatedTimeInMs': '4685281000'}}
n29576362 {'freshness': {'numberMissedRelease': '367', 'outdatedTimeInMs': '142773884000'}}
n29576364 {'freshness': {'numberMissedRelease': '448', 'outdatedTimeInMs': '105191360000'}}
n29576366 {'freshness': {'numberMissedRelease': '365', 'outdatedTimeInMs': '142745787000'}}
n29576368 {'freshness': {'numberMissedRelease': '3', 'outdatedTimeInMs': '6591247000'}}
n29576370 {'freshness': {'numberMissedRelease': '0', 'outdatedTimeInMs': '0'}}
n29576372 {'freshness': {'numberMissedRelease': '5', 'outdatedTimeInMs': '3391477000'}}
n29576374 {'freshness': {'numberMissedRelease': '1', 'outdatedTimeInMs': '117088000'}}


## process popularity value

In [27]:
# Filter nodes that have the 'severity' attribute
nodes_with_popu = {node_id: node for node_id, node in nodes.items() if 'type' in node and node['type'] == "POPULARITY_1_YEAR" and str_to_json(node["value"]) !='0'}

In [28]:
for node_id, node in list(nodes_with_popu.items())[:10]:
    print(node_id, node["value"])
    print(node_id, node)

n29576357 0
n29576357 {'labels': ':AddedValue', 'value': '0', 'id': 'com.softwaremill.sttp.client:core_sjs0.6_2.13:2.0.2:POPULARITY_1_YEAR', 'type': 'POPULARITY_1_YEAR'}
n29576359 0
n29576359 {'labels': ':AddedValue', 'value': '0', 'id': 'com.ibeetl:act-sample:3.0.0-M6:POPULARITY_1_YEAR', 'type': 'POPULARITY_1_YEAR'}
n29576361 0
n29576361 {'labels': ':AddedValue', 'value': '0', 'id': 'com.softwaremill.sttp.client:core_sjs0.6_2.13:2.0.0:POPULARITY_1_YEAR', 'type': 'POPULARITY_1_YEAR'}
n29576363 0
n29576363 {'labels': ':AddedValue', 'value': '0', 'id': 'com.lihaoyi:ammonite_2.12.1:0.9.8:POPULARITY_1_YEAR', 'type': 'POPULARITY_1_YEAR'}
n29576365 0
n29576365 {'labels': ':AddedValue', 'value': '0', 'id': 'com.yahoo.vespa:container-disc:7.394.21:POPULARITY_1_YEAR', 'type': 'POPULARITY_1_YEAR'}
n29576367 0
n29576367 {'labels': ':AddedValue', 'value': '0', 'id': 'com.lihaoyi:ammonite_2.12.1:0.9.9:POPULARITY_1_YEAR', 'type': 'POPULARITY_1_YEAR'}
n29576369 0
n29576369 {'labels': ':AddedValue', '

## process speed value

In [25]:
# Filter nodes that have the 'severity' attribute
nodes_with_speed = {node_id: node for node_id, node in nodes.items() if 'type' in node and node['type'] == "SPEED" and str_to_json(node["value"]) !='0'}

In [26]:
for node_id, node in list(nodes_with_speed.items())[:10]:
    print(node_id, node["value"])
    print(node_id, node)

n58494634 0.08070175438596491
n58494634 {'labels': ':AddedValue', 'id': 'se.fortnox.reactivewizard:reactivewizard-jaxrs:SPEED', 'type': 'SPEED', 'value': '0.08070175438596491'}
n58494635 0.08564476885644769
n58494635 {'labels': ':AddedValue', 'id': 'com.walmartlabs.concord.plugins.basic:http-tasks:SPEED', 'type': 'SPEED', 'value': '0.08564476885644769'}
n58494636 0.029411764705882353
n58494636 {'labels': ':AddedValue', 'id': 'cc.akkaha:asura-dubbo_2.12:SPEED', 'type': 'SPEED', 'value': '0.029411764705882353'}
n58494637 0.04967197750702906
n58494637 {'labels': ':AddedValue', 'id': 'org.wildfly.security:wildfly-elytron-jose-jwk:SPEED', 'type': 'SPEED', 'value': '0.04967197750702906'}
n58494638 0.3325480957989792
n58494638 {'labels': ':AddedValue', 'id': 'com.yahoo.vespa:container-messagebus:SPEED', 'type': 'SPEED', 'value': '0.3325480957989792'}
n58494639 0.014814814814814815
n58494639 {'labels': ':AddedValue', 'id': 'it.tidalwave.thesefoolishthings:it-tidalwave-thesefoolishthings-exampl